In [13]:
#%%file ffsr.py
""" Pseudocode for Fast FSR algorithm """

""" Date: 4/8/15 
    Modified: Profile primary function ffsr """

""" Date: 4/17/15
    Modified: Corrected p-value function, p-mono corrected """

""" Date: 4/19/15
    Modified: Corrected gamma_F function to correctly handle duplicate p-values """


""" Data type check """
def df_type(dat):
    
    ### Input params:
    #   dat = dataset whose type is to be checked / transformed
    
    ### Output:
    #   error msg or True boolean
    
    import numpy as np
    import pandas as pd
    
    if isinstance(dat,pd.DataFrame)==False and isinstance(dat,np.ndarray)==False:
        raise Exception("Data must be pandas DataFrame")
    else:
        return True


    
""" p-value computation function """
def pval_comp(max_size=None):
    
    ### Input params:
    #   max_size = integer max no. of vars in final model (largest model size desired)
    
    ### Output:
    # array of p-values of each covariate at its given entry step
    
    ### NOTE: fwd should be a global-env R object (requires running 'forward' fcn prior to this fcn) ###
    
    import numpy as np
    import scipy.stats as st
    import rpy2.robjects as ro
    
    # Pull RSS values & num_obs from fwd_proc object
    rss = np.array(ro.r('fwd$rss'))
    N = np.array(ro.r('fwd$nn'))
    
    if max_size==None:
        max_size = len(rss)-1
    
    # vector of model sizes
    sizes = np.arange(max_size)+1
    
    # compute the F stats as defined above where p_f - p_r = 1 for each iteration
    fstats = (rss[0:max_size] - rss[1:(max_size+1)]) / (rss[1:(max_size+1)] / (N - (sizes+1)))
    
    # return the p-values by comparing these stats to the F distn: F(1, n - p_f)
    return 1 - st.f.cdf(fstats, 1, N-(sizes+1))



""" Covariate model entry order """
def cov_order(xcolnames,max_size=None,col_incl=None):
    
    # Input params:
    #   xcolnames = array of names of covariates (same order as columns in original dataset)
    #   max_size  = integer max no. of vars in final model (largest model size desired)
    #   col_incl  = array vector of columns to forcefully include in all models
    
    ### Output:
    # array of covariate names sorted according to order of entry into the model
    
    ### NOTE: fwd should be a global-env R object (requires running 'forward' fcn prior to this fcn) ###
    
    import numpy as np
    import rpy2.robjects as ro
    
    if max_size==None:
        max_size = len(xcolnames)
        
    ### Pull the cov entry order
    vorder = ro.r('fwd$vorder[-1]') # remove intercept
    vorder = vorder[0:max_size] # keep only the max model size number of covs
    
    ### Shift these values down by two (one to exclude intercept, one to make python indices)
    vorderinds = np.array(vorder)-2
    
    ### Rearrange the var order st forced vars are at start of list
    if col_incl==None:
        col_incl = np.arange(max_size)+1
    keep = xcolnames[[col_incl-1]] # pull var names of those vars forced into model (this is an array)
    poss = [x for x in xcolnames if x not in keep] # pull var names of those not forced in (this is a list)
    col_names = np.array(list(keep)+poss) # = rearranged array of varnames w/forced-in vars at start of list
    
    ### Sort the columns of X in order to obtain the var names in the entry order
    return col_names[vorderinds[::]]

    

""" Forward selection function """
def forward(x,y,max_size=None,col_incl=None):
    
    ### Input params:
    #   x        = python dataframe of original p covariates, n x p
    #   y        = python outcome dataframe, n x 1
    #   max_size = integer max no. of vars in final model (largest model size desired)
    #   col_incl = array vector of columns to forcefully include in all models
    
    ### Output:
    # regsubsets R object -- the raw full output of the forward selection proc
    
    ### Load python packages to call R functions
    import rpy2.robjects as ro
    import pandas.rpy.common as com
    
    ### Convert x and y to R matrices <-- MAKE SURE x,y input == DATAFRAMES (or else change them to df's)!!!
    ### and declare as R objects in global environment
    ro.globalenv['x2'] = com.convert_to_r_matrix(x)
    ro.globalenv['y2'] = com.convert_to_r_matrix(y)
    if max_size==None:
        max_size = x.shape[1]
    ro.globalenv['maxv'] = ro.Vector(max_size)
    if col_incl==None:
        ro.r('coli=NULL')
    else:
        ro.globalenv['coli'] = ro.FloatVector(col_incl[:])
    
    ### Perform forward selection with regsubsets function
    ro.globalenv['fwd'] = ro.r('leaps::regsubsets(x=x2,y=y2,method="forward",nvmax=maxv,force.in=coli)')
    
    
    
""" Gamma computation """
def gamma_F(pvs, ncov, max_size=None):
    
    ### Input params:
    #   pvs      = vector of p-values (sorted or unsorted) from forward sel procedure
    #   ncov     = integer total number of covariates in data
    #   max_size = integer max no. of vars in final model (largest model size desired)
    
    ### Output:
    # array of gamma_F values
    
    import numpy as np
    
    if max_size==None:
        max_size = ncov
        
    # Create indices == model size at given step, call this S
    S = np.arange(max_size)+1
    
    # gamma_F_i = p_s_i * (ncov - S_i) / (1 + S_i)
    g_F = pvs * (ncov - S) / (1 + S)
    
    # Check for duplicate p-values
    dups = list(set([x for x in list(pvs) if list(pvs).count(x) > 1]))
    for i in range(len(dups)): g_F[pvs==dups[i]] = min(g_F[pvs==dups[i]])
    
    # if table run on all vars, the last gamma = 0,
    #  instead set equal to the last pv_sort == final rate of unimp var inclusion
    if(g_F[-1]==0): 
        g_F[-1]=pvs[-1]
    
    return g_F

    
    
""" Alpha computation for model selection """
def alpha_F(g0, ncov, max_size=None):
    
    ### Input params:
    #   g0       = float pre-specified FSR (gamma0)
    #   ncov     = integer total number of covariates in data
    #   max_size = integer max no. of vars in final model (largest model size desired)
    
    ### Output:
    # array of alpha_F values
    
    import numpy as np
    
    if max_size==None:
        max_size = ncov
        
    # Create indices == model size at given step, call this S
    S = np.arange(max_size)+1
    
    # alpha_F_i = gamma_0 * (1 + S_i) / (ncov - S_i)
    alpha_F = g0 * (1 + S) / (ncov - S)
    
    # if table run on all vars, the last alpha = inf
    #  instead set equal to 1 == include all vars
    alpha_F[np.isinf(alpha_F)] = 1.
    
    return alpha_F        
    
    
    
""" Alpha computation for specific gamma """
def alpha_F_g(g, gf, ncov):
    
    ### Input params:
    #   g    = float or vector (length k) of specified FSR at which to compute alpha
    #   gf   = vector gamma_F's computed from gamma0, pv_sorted
    #          used to compute largest size model (S) for which gamma_F < g
    #   ncov = integer of total number covariates in data
    
    ### Output:
    # integer alpha_F value
    
    import numpy as np
    
    ### Compute model size for gf closest to (but still <) g
    #S = np.array([max(np.which(x<=y)) for x in gf y in g])+1
    if isinstance(g,np.ndarray): # if g is a vector
        s_s = [np.where(gf>y) for y in g]
        S = np.array([min(x[0]) for x in s_s])
        return g * (1 + S) / (ncov - S)
    else: # if g is a number
        S = min(np.where(gf>g)[0])
        return g * (1 + S) / (ncov - S)


    
""" Beta-hat computation for specific gamma """
def beta_est(x, y, g, gf, vname):
    
    ### Input params:
    #   x      = python dataframe of original p covariates, n x p
    #   y      = python outcome dataframe, n x 1
    #   g      = float of specified FSR at which to compute alpha
    #   gf     = vector gamma_F's computed from gamma0, pv_sorted
    #            used to compute largest size model (S) for which gamma_F < g
    #   vname  = ordered vector of names of vars entered into model under forward selection
    
    ### Output:
    # array of estimated parameters
    
    import numpy as np
    import statsmodels.api as sm
    
    ### Compute model size corresponding to g
    S = min(np.where(gf>g)[0])

    ### Pull the cov names of those vars included in the above size model
    modvars = vname[:S]

    ### Fit the linear model using the selected model vars
    fit = sm.OLS(y,x.loc[:,list(modvars)]).fit()
    betaout = pd.DataFrame([fit.params,fit.bse]).T
    betaout.columns = ['beta','beta_se']
    
    return betaout

    
    
""" FSR Results Table """
def fsrtable(size, vname, p_orig, p_mono, alphaf, gammaf, prec_f=4):
    
    ### Input params:
    #   size   = model size at each step of forward sel proc                   [S]
    #   vname  = variable name that entered at each step (num vars = p)        [Var]
    #   p_orig = p-values at each step                                         [p]
    #   p_mono = ascending p-values                                            [p_s]
    #   alphaf = alpha-to-enter (p-value cutoff) for model entry at each step  [alpha_F]
    #   gammaf = FSR at each step                                              [gamma_F]
    #   prec_f   = integer of precision (num digits) desired in FSR output table
    
    ### Output:
    # table of [S   Var   p   p_s   alpha_F   gamma_F], dim = num_steps(== p) x 6
    
    import pandas as pd
    
    ### Convert all arrays to dataframes
    sized = pd.DataFrame(size)
    vnamed = pd.DataFrame(vname)
    p_od = pd.DataFrame(np.around(p_orig,prec_f))
    p_sd = pd.DataFrame(np.around(p_mono,prec_f))
    ad = pd.DataFrame(np.around(alphaf,prec_f))
    gd = pd.DataFrame(np.around(gammaf,prec_f))
    
    ### Combine the arrays
    tab = pd.concat([sized,vnamed,p_od,p_sd,ad,gd],axis=1)
    tab.columns = ['S','Var','p','p_s','alpha_F','gamma_F']
    
    return tab
    
    
    
""" FastFSR function """
def ffsr(X,Y,g0=0.05,betaout=False,gs=None,max_size=None,var_incl=None,bag=False,prec_f=4,prec_b=6):
    
    ### Input params:
    #   x        = python dataframe of original p covariates, n x p
    #   y        = python outcome dataframe, n x 1
    #   g0       = float pre-specified FSR of interest ("gamma0")
    #   betaout  = boolean of whether to include estimated betahats from final selected model
    #   gs       = float or vector of gamma's at which to specifically compute alpha_F
    #   max_size = integer of largest model size == max num vars to incl in final model (default = num covs in dataset)
    #   var_incl = array of cols corresponding to those vars to force into model
    #   bag      = boolean of whether to output FSR table (non-bagging results) or reduced output for bagging purposes
    #   prec_f   = integer of precision (num digits) desired in FSR output table
    #   prec_b   = integer of precision (num digits) desired in beta-hat parameter estimates of final model
    
    ### Output: 
    #      (note: gamma = FSR, gamma_0 = pre-specified/desired FSR)
    # Table of [S   Var   p   p_s   alpha_F   gamma_F], dim = num_steps(== p) x 6
    #   S:       model size at given step
    #   Var:     name of var that entered at given step
    #   p:       p-value of var that entered at given step
    #   p_s:     sorted p-value (vector or original p-values sorted in increasing order)
    #   alpha_F: cutoff value for model entry given gamma_0 and current p_s value
    #   gamma_F: FSR given current alpha_F and model size (== step num)
    #       and
    # Vector of alpha_F's for specified gamma's (g)

    import numpy as np
    import pandas as pd
    
    ### Clean and check data - make sure X, Y = pandas dataframes or else convert them
    if df_type(X)==True:
        if isinstance(X,pd.DataFrame):
            x = X.copy()
        else:
            if isinstance(X,np.ndarray):
                x = pd.DataFrame(X)
                vnum = list(np.arange(x.shape[1])+1)
                vchr = list(np.repeat("V",x.shape[1]))
                x.columns = [a + str(b) for a,b in zip(vchr,vnum)]
    else:
        return df_type(X)
    if df_type(Y)==True:
        if isinstance(Y,pd.DataFrame):
            y = Y.copy()
        else:
            if isinstance(Y,np.ndarray):
                y = pd.DataFrame(Y)
    else:
        return df_type(Y)
    
    # remove missing values
    yna = np.isnan(y).any(axis=1)
    xna = np.isnan(x).any(axis=1).reshape(x.shape[0],1)
    anyna = np.array([int(max(a,b)) for a,b in zip(xna,yna)])
    missrow = np.where(anyna==1)[0]
    y = y.drop(y.index[missrow])
    x = x.drop(x.index[missrow])
    
    # check that p < n to ensure regression solutions
    if x.shape[1] >= x.shape[0]:
        raise Exception("N must be > p for valid regression solutions")
    
    ### If max model size not specified, select all possible cov.s
    if max_size==None:
        max_size = x.shape[1]
        
    ### Perform forward selection
    fwd_sel = forward(x, y, max_size, var_incl)
    
    ### Save order of covariate entry into model
    cov_entry_order = cov_order(x.columns.values, max_size, var_incl)
    
    ### Compute p-value of each covariate entering the model
    p_orig = pval_comp(max_size)
    
    ### Sort p-values in ascending order
    p_sort = np.array([max(p_orig[:(i+1)]) for i in range(len(p_orig))])
        
    ### Gamma_F computation
    g_F = gamma_F(p_sort, x.shape[1], max_size)
    
    ### Check if betaout desired, if so compute beta_hat of model corresponding to specific gamma0
    if betaout==True or bag==True:
        betahats = beta_est(x, y, g0, g_F, cov_entry_order)
        
    ### Check if bagging desired
    if bag==False: 
        ### Alpha_F computation for all steps in fwd sel proc
        a_F = alpha_F(g0, x.shape[1], max_size)
        
        ### Model size
        S = np.arange(max_size)+1
        
        ### Combine S, Cov_names, p-vals, sorted p-vals, alpha_F, gamma_F into table
        fsr_results = fsrtable(S, cov_entry_order, p_orig, p_sort, a_F, g_F)
        
        ### Return selected output: FSR table (+ betahat) (+ alpha_specific)
        if gs!=None: 
            ### Compute alpha_F for specific gammas (gs)
            if betaout==True:
                return fsr_results, np.around(betahats, prec_b), alpha_F_g(gs, g_F, x.shape[1])
            else:
                return fsr_results, alpha_F_g(gs, g_F, x.shape[1])
        else:
            if betaout==True:
                return fsr_results, np.around(betahats, prec_b)
            else:
                return fsr_results
    else:
        return betahats, alpha_F_g(g0, g_F, x.shape[1]), len(betahats)

    
    
def bagfsr(X,Y,g0,B=200,max_s=None,v_incl=None,prec=4):
    
    ### Input params:
    #   X      = python dataframe of original p covariates, n x p
    #   Y      = python outcome dataframe, n x 1
    #   g0     = float pre-specified FSR of interest ("gamma0")
    #   B      = integer of number of bagged samples
    #   max_s  = integer of largest model size == max num vars to incl in final model (default = num covs in dataset)
    #   v_incl = array of cols corresponding to those vars to force into model
    #   prec   = integer of precision (num digits) desired in beta-hat parameter estimates of final model
    
    ### Output: 
    #   Mean of betahats
    #   SEs of betahats
    #   Avg alpha-to-enter
    #   Avg model size
    #   Prop of times each var included in model
    
    import numpy as np
    import pandas as pd
    
    ### Clean and check data - make sure X, Y = pandas dataframes or else convert them
    if df_type(X)==True:
        if isinstance(X,pd.DataFrame):
            x = X.copy()
        else:
            if isinstance(X,np.ndarray):
                x = pd.DataFrame(X)
                vnum = list(np.arange(x.shape[1])+1)
                vchr = list(np.repeat("V",x.shape[1]))
                x.columns = [a + str(b) for a,b in zip(vchr,vnum)]
    else:
        return df_type(X)
    if df_type(Y)==True:
        if isinstance(Y,pd.DataFrame):
            y = Y.copy()
        else:
            if isinstance(Y,np.ndarray):
                y = pd.DataFrame(Y)
    else:
        return df_type(Y)
    
    ### Combine data into single dataframe
    dat = pd.concat([y,x],axis=1)
    
    ### Create array to keep track of number of times vars enter model
    nentries = pd.DataFrame(np.zeros(x.shape[1]),index=x.columns.values)
    
    ### Create array to store all estimated coefficients, ses, alphas, sizes
    allbetas = pd.DataFrame(np.zeros([B,x.shape[1]]),columns=x.columns.values)
    allses = allbetas.copy()
    alphas = []
    sizes = []
    
    ### Bagging loops
    for i in range(B):

        # Draw with replacement from rows of data
        np.random.seed(1234)
        n_row = dat.shape[0]
        rand_row = np.random.randint(0,n_row,n_row)
        newdat = dat.iloc[rand_row,:]
        newdat.index = np.arange(n_row)+1
        
        ### Obtain FSR results
        prec_all = 8 # precision of output
        fsrout = ffsr(newdat.iloc[:,1:],pd.DataFrame(newdat.iloc[:,0]),g0,bag=True,max_size=max_s,var_incl=v_incl)
        allbetas.loc[i,fsrout[0].index.values] = np.around(fsrout[0].iloc[:,0],prec_all)
        allses.loc[i,fsrout[0].index.values] = np.around(fsrout[0].iloc[:,1],prec_all)
        alphas.append(fsrout[1])
        sizes.append(fsrout[2])

        ### Update counts num times var included
        nentries.loc[fsrout[0].index[np.abs(np.around(fsrout[0].iloc[:,0],prec_all))>0]] += 1
        
    ### Compute averages
    avgbeta = allbetas.mean(axis=0) # mean across rows / colmeans == mean of each cov's betahat
    avgse = allses.mean(axis=0)
    avgalpha = np.mean(alphas)
    avgsize = np.mean(sizes)
    var_props = nentries/float(B)
    cov_res = pd.concat([avgbeta,avgse,var_props],axis=1)
    cov_res.columns = ['betahat','betase','prop_incl']
    
    return cov_res, avgalpha, avgsize
    
    

# Notes: 
# 1. appropriate transformations are expected to have been applied prior to utilization of FSR algorithm

# To-do:
# 1. adjust betaest fcn and ffsr to allow for specification of intercept and whether data should be normalized in estimation

In [14]:
import numpy as np
import pandas as pd

np.random.seed(1234)

X = np.random.multivariate_normal(np.zeros(15),np.eye(15),(100))
beta = np.array([0.,0,5,6,0,0,4,0,0,0,5,0,0,0,0]).reshape(15,1) # signif betas: 3,4,7,11
Y = X.dot(beta)

In [15]:
Y2 = pd.DataFrame(Y)
X2 = pd.DataFrame(X)
X2.columns = ["V1","V2","V3","V4","V5","V6","V7","V8","V9","V10","V11","V12","V13","V14","V15"]

In [37]:
def seq_forw_select(features, max_k, criterion_func, print_steps=False):
    """
    Implementation of a Sequential Forward Selection algorithm.
    
    Keyword Arguments:
        features (list): The feature space as a list of features.
        max_k: Termination criterion; the size of the returned feature subset.
        criterion_func (function): Function that is used to evaluate the
            performance of the feature subset.
        print_steps (bool): Prints the algorithm procedure if True.
    
    Returns the selected feature subset, a list of features of length max_k.

    """
    
    # Initialization
    feat_sub = []
    k = 0
    d = len(features)
    if max_k > d:
        max_k = d
    
    while True:
        
        # Inclusion step
        if print_steps:
            print('\nInclusion from feature space', features)
        crit_func_max = criterion_func(feat_sub + [features[0]])
        best_feat = features[0]
        for x in features[1:]:
            crit_func_eval = criterion_func(feat_sub + [x])
            if crit_func_eval > crit_func_max:
                crit_func_max = crit_func_eval
                best_feat = x
        feat_sub.append(best_feat)
        if print_steps:
            print('include: {} -> feature subset: {}'.format(best_feat, feat_sub))
        features.remove(best_feat)
        
        # Termination condition
        k = len(feat_sub)
        if k == max_k:
            break
                
    return feat_sub

In [ ]:
### Comparisons with R code

In [16]:
%load_ext rpy2.ipython

In [22]:
%%R -i X2,Y2

bag.fsr<-function(x,y,B=100,gam0=.05){
# gives average coefficients from fsr.fast6.sim
ok<-complete.cases(x,y)
x<-x[ok,]                            # get rid of na's
y<-y[ok]                             # since regsubsets can't handle na's
m<-ncol(x)
n<-nrow(x)
hold<-matrix(rep(0,m*B),nrow=B)      # holds coefficients
interc<-rep(0,B)                     # holds intercepts
alphahat<-rep(0,B)                   # holds alphahats
size<-rep(0,B)                       # holds sizes
for(i in 1:B){
index<-sample(1:n,n,replace=T)
out<-fsr.fast6.sim(x=x[index,],y=y[index],gam0=gam0)
if (out$size>0) hold[i,out$x.ind]<-out$mod$coeff[2:(out$size+1)]
interc[i]<-out$mod$coeff[1]
alphahat[i]<-out$alphahat.ER
size[i]<-out$size
}                                    # ends i loop
coeff.av<-apply(hold,2, mean)
coeff.sd<-rep(0,m)
coeff.sd<-sqrt(apply(hold,2, var))
interc.av<-mean(interc)
interc.sd<-sd(interc)
amean<-mean(alphahat)
sizem<-mean(size)
prop<-rep(0,m)
for(j in 1:m){prop[j]<-sum(abs(hold[,j])>0)/B}
as.matrix(x)->x                      # in case x is a data frame
pred<-x%*%coeff.av+interc.av
return(list(coeff.av=coeff.av,coeff.sd=coeff.sd,interc.av=interc.av,pred=pred,
            interc.sd=interc.sd,prop=prop,amean=amean,sizem=sizem))
}

fsr.fast6.sim<-function(x,y,gam0=.05){
# estimated alpha for forward selection
# short output version
require(leaps)
ok<-complete.cases(x,y)
x<-x[ok,]                            # get rid of na's
y<-y[ok]                             # since regsubsets can't handle na's
m<-ncol(x)
n<-nrow(x)
if(m >= n) m1 <- n-5  else m1<-m     # to get rid of NA's in pv
vm<-1:m1
as.matrix(x)->x                      # in case x is a data frame
pvm<-rep(0,m1)                       # to create pvm below
regsubsets(x,y,method="forward")->out.x
pv.orig<-1-pf((out.x$rss[vm]-out.x$rss[vm+1])*(out.x$nn-(vm+1))/out.x$rss[vm+1],1,out.x$nn-(vm+1))
for (i in 1:m1){pvm[i]<-max(pv.orig[1:i])}  # sequential max of pvalues
alpha<-c(0,pvm)
ng<-length(alpha)
S<-rep(0,ng)                         # will contain num. of true entering in orig.
real.seq<-data.frame(var=(out.x$vorder-1)[2:(m1+1)],pval=pv.orig,
         pvmax=pvm,Rsq=round(1-out.x$rss[2:(m1+1)]/out.x$rss[1],4))
for (ia in 2:ng){                    # loop through alpha values for S=size
S[ia] <- sum(pvm<=alpha[ia])         # size of models at alpha[ia], S[1]=0
}
ghat<-(m-S)*alpha/(1+S)              # gammahat_ER
# add additional points to make jumps
alpha2<-alpha[2:ng]-.0000001
ghat2<-(m-S[1:(ng-1)])*alpha2/(1+S[1:(ng-1)])
zp<-data.frame(a=c(alpha,alpha2),g=c(ghat,ghat2))
zp<-zp[order(zp$a),]
gmax<-max(zp$g)
index.max<-which.max(zp$g)           # index of largest ghat
alphamax<-zp$a[index.max]            # alpha with largest ghat
ind<-(ghat <= gam0 & alpha<=alphamax)*1
Sind<-S[max(which(ind > 0))]          # model size with ghat just below gam0
alphahat.fast<-(1+Sind)*gam0/(m-Sind)  # ER est.
size1<-sum(pvm<=alphahat.fast)+1       # size of model including intercept
colnames(x)<-colnames(x,do.NULL=F,prefix="")      # corrects for no colnames
x<-x[,colnames(x)[(out.x$vorder-1)[2:size1]]]
if(size1>1) x.ind<-(out.x$vorder-1)[2:size1]  else x.ind<-0
if (size1==1) {mod <- lm(y~1)} else {mod <- lm(y~x)}
return(list(mod=mod,size=size1-1,x.ind=x.ind,alphahat.ER=alphahat.fast))
}
    
x2 = as.matrix(X2)
y2 = as.matrix(Y2)

system.time(bag.fsr(x=x2,y=y2,B=200)->outt)

Loading required package: leaps
   user  system elapsed 
  0.614   0.007   0.635 


In [11]:
%%time 
b = bagfsr(X2,Y2,0.05) # <-- TOO SLOW! see future drafts for edits to bagging

CPU times: user 4.73 s, sys: 81.7 ms, total: 4.82 s
Wall time: 4.84 s


In [23]:
""" FastFSR function """
def ffsr(X,Y,g0=0.05,betaout=False,gs=None,max_size=None,var_incl=None,bag=False,prec_f=4,prec_b=6):
    
    ### Input params:
    #   x        = python dataframe of original p covariates, n x p
    #   y        = python outcome dataframe, n x 1
    #   g0       = float pre-specified FSR of interest ("gamma0")
    #   betaout  = boolean of whether to include estimated betahats from final selected model
    #   gs       = float or vector of gamma's at which to specifically compute alpha_F
    #   max_size = integer of largest model size == max num vars to incl in final model (default = num covs in dataset)
    #   var_incl = array of cols corresponding to those vars to force into model
    #   bag      = boolean of whether to output FSR table (non-bagging results) or reduced output for bagging purposes
    #   prec_f   = integer of precision (num digits) desired in FSR output table
    #   prec_b   = integer of precision (num digits) desired in beta-hat parameter estimates of final model
    
    ### Output: 
    #      (note: gamma = FSR, gamma_0 = pre-specified/desired FSR)
    # Table of [S   Var   p   p_s   alpha_F   gamma_F], dim = num_steps(== p) x 6
    #   S:       model size at given step
    #   Var:     name of var that entered at given step
    #   p:       p-value of var that entered at given step
    #   p_m:     mono. inc. p-value (vector or original p-values arranged to be monotonically increasing)
    #   alpha_F: cutoff value for model entry given gamma_0 and current p_s value
    #   gamma_F: FSR given current alpha_F and model size (== step num)
    #       and
    #   vector of alpha_F's for specified gamma's (g)
    #       and
    #   vector of estimated beta param's for final model (based on g0)

    import numpy as np
    import pandas as pd
    
    ### Clean and check data - make sure X, Y = pandas dataframes or else convert them
    if bag==False:
        if df_type(X)==True:
            if isinstance(X,pd.DataFrame):
                x = X.copy()
            else:
                if isinstance(X,np.ndarray):
                    x = pd.DataFrame(X)
                    vnum = list(np.arange(x.shape[1])+1)
                    vchr = list(np.repeat("V",x.shape[1]))
                    x.columns = [a + str(b) for a,b in zip(vchr,vnum)]
        else:
            return df_type(X)
        if df_type(Y)==True:
            if isinstance(Y,pd.DataFrame):
                y = Y.copy()
            else:
                if isinstance(Y,np.ndarray):
                    y = pd.DataFrame(Y)
        else:
            return df_type(Y)
        # Remove missing values
        yna = np.isnan(y).any(axis=1)
        xna = np.isnan(x).any(axis=1).reshape(x.shape[0],1)
        anyna = np.array([int(max(a,b)) for a,b in zip(xna,yna)])
        missrow = np.where(anyna==1)[0]
        y = y.drop(y.index[missrow])
        x = x.drop(x.index[missrow])
        # Check that p < n to ensure regression solutions
        if x.shape[1] >= x.shape[0]:
            raise Exception("N must be > p for valid regression solutions")
    else:
        x, y = X.copy(), Y.copy()
    
    ### If max model size not specified, select all possible cov.s
    if max_size==None:
        max_size = x.shape[1]
        
    ### Perform forward selection
    fwd_sel = forward(x, y, max_size, var_incl)
    
    ### Save order of covariate entry into model
    cov_entry_order = cov_order(x.columns.values, max_size, var_incl)
    
    ### Compute p-value of each covariate entering the model
    p_orig = pval_comp(max_size)
    
    ### Arrange p-values in mono. inc. order
    p_mono = np.array([max(p_orig[:(i+1)]) for i in range(len(p_orig))])
        
    ### Gamma_F computation
    g_F = gamma_F(p_mono, x.shape[1], max_size)
    
    ### Check if betaout desired, if so compute beta_hat of model corresponding to specific gamma0
    if betaout==True or bag==True:
        betahats = beta_est(x, y, g0, g_F, cov_entry_order)
        
    ### Check if bagging desired
    if bag==False: 
        ### Alpha_F computation for all steps in fwd sel proc
        a_F = alpha_F(g0, x.shape[1], max_size)
        
        ### Model size
        S = np.arange(max_size)+1
        
        ### Combine S, Cov_names, p-vals, sorted p-vals, alpha_F, gamma_F into table
        fsr_results = fsrtable(S, cov_entry_order, p_orig, p_mono, a_F, g_F)
        
        ### Return selected output: FSR table (+ betahat) (+ alpha_specific)
        if gs!=None: 
            ### Compute alpha_F for specific gammas (gs)
            if betaout==True:
                return fsr_results, np.around(betahats, prec_b), alpha_F_g(gs, g_F, x.shape[1])
            else:
                return fsr_results, alpha_F_g(gs, g_F, x.shape[1])
        else:
            if betaout==True:
                return fsr_results, np.around(betahats, prec_b)
            else:
                return fsr_results
    else:
        return betahats, alpha_F_g(g0, g_F, x.shape[1]), len(betahats)

    
    
def bagfsr(X,Y,g0,B=200,max_s=None,v_incl=None,prec=4):
    
    ### Input params:
    #   X      = python dataframe of original p covariates, n x p
    #   Y      = python outcome dataframe, n x 1
    #   g0     = float pre-specified FSR of interest ("gamma0")
    #   B      = integer of number of bagged samples
    #   max_s  = integer of largest model size == max num vars to incl in final model (default = num covs in dataset)
    #   v_incl = array of cols corresponding to those vars to force into model
    #   prec   = integer of precision (num digits) desired in beta-hat parameter estimates of final model
    
    ### Output: 
    #   Mean of betahats
    #   SEs of betahats
    #   Avg alpha-to-enter
    #   Avg model size
    #   Prop of times each var included in model
    
    import numpy as np
    import pandas as pd
    
    ### Clean and check data - make sure X, Y = pandas dataframes or else convert them
    if df_type(X)==True:
        if isinstance(X,pd.DataFrame):
            x = X.copy()
        else:
            if isinstance(X,np.ndarray):
                x = pd.DataFrame(X)
                vnum = list(np.arange(x.shape[1])+1)
                vchr = list(np.repeat("V",x.shape[1]))
                x.columns = [a + str(b) for a,b in zip(vchr,vnum)]
    else:
        return df_type(X)
    if df_type(Y)==True:
        if isinstance(Y,pd.DataFrame):
            y = Y.copy()
        else:
            if isinstance(Y,np.ndarray):
                y = pd.DataFrame(Y)
    else:
        return df_type(Y)
    
    # Remove missing values
    yna = np.isnan(y).any(axis=1)
    xna = np.isnan(x).any(axis=1).reshape(x.shape[0],1)
    anyna = np.array([int(max(a,b)) for a,b in zip(xna,yna)])
    missrow = np.where(anyna==1)[0]
    y = y.drop(y.index[missrow])
    x = x.drop(x.index[missrow])
    
    # check that p < n to ensure regression solutions
    if x.shape[1] >= x.shape[0]:
        raise Exception("N must be > p for valid regression solutions")
    
    ### Combine data into single dataframe
    dat = pd.concat([y,x],axis=1)
    
    ### Create array to keep track of number of times vars enter model
    nentries = pd.DataFrame(np.zeros(x.shape[1]),index=x.columns.values)
    
    ### Create array to store all estimated coefficients, ses, alphas, sizes
    allbetas = pd.DataFrame(np.zeros([B,x.shape[1]]),columns=x.columns.values)
    allses = allbetas.copy()
    alphas = []
    sizes = []
    
    np.random.seed(1234)
    
    ### Bagging loops
    for i in range(B):

        # Draw with replacement from rows of data
        n_row = dat.shape[0]
        rand_row = np.random.randint(0,n_row,n_row)
        newdat = dat.iloc[rand_row,:]
        newdat.index = np.arange(n_row)+1
        
        ### Obtain FSR results
        fsrout = ffsr(newdat.iloc[:,1:],pd.DataFrame(newdat.iloc[:,0]),g0,bag=True,max_size=max_s,var_incl=v_incl)
        allbetas.loc[i,fsrout[0].index.values] = fsrout[0].iloc[:,0]
        allses.loc[i,fsrout[0].index.values] = fsrout[0].iloc[:,1]
        alphas.append(fsrout[1])
        sizes.append(fsrout[2])

        ### Update counts num times var included
        nentries.loc[fsrout[0].index[np.abs(np.around(fsrout[0].iloc[:,0],prec))>0]] += 1
        
    ### Compute averages
    avgbeta = np.around(allbetas.mean(axis=0),prec) # mean across rows / colmeans == mean of each cov's betahat
    avgse = np.around(allses.mean(axis=0),prec)
    avgalpha = np.mean(alphas)
    avgsize = np.mean(sizes)
    var_props = nentries/float(B)
    cov_res = pd.concat([avgbeta,avgse,var_props],axis=1)
    cov_res.columns = ['betahat','betase','prop_incl']
    
    return cov_res, avgalpha, avgsize

In [24]:
%%time 
bagfsr(X2,Y2,0.05) # <-- TOO SLOW! see future drafts for edits to bagging

CPU times: user 4.33 s, sys: 70.2 ms, total: 4.4 s
Wall time: 4.43 s


(     betahat  betase  prop_incl
 V1         0       0          0
 V2         0       0          0
 V3         5       0          1
 V4         6       0          1
 V5         0       0          0
 V6        -0       0          0
 V7         4       0          1
 V8         0       0          0
 V9        -0       0          0
 V10        0       0          0
 V11        5       0          1
 V12        0       0          0
 V13        0       0          0
 V14       -0       0          0
 V15       -0       0          0, 0.049823376623376624, 6.1449999999999996)

In [25]:
# comparing the above R result with %%time result of my algorithm shows the following increase:
# user: + 4.12s,  sys: + 7.07ms,  elapsed: + 4.192ms

In [ ]:
###########################################################
# OPTIMIZATION PLAN:

"""  
NOTE: numba, numbapro do not work on (any) some of my FSR functions due to incompatibility w/list comprehension (among other things)
note to self: write subfunction to remove missing values and attempt following optimization strategies

1. Try to vectorize additional lines of code to improve speed/efficiency
2. Try to translate some functions into C-code (perhaps using numexpr), or else find c equivalents which would be faster than the 
   given python versions
    a. This should be viable for those functions that do not manipulate R objects
    b. Perhaps search for a C/C++ forward selection function (likely faster than regsubsets which I am calling from R)
3. Closely examine code to determine where memory is needlessly being used (e.g. are duplicates being made unnecessarily)
"""